# Lending Club Case Study

## Problem statement:
If I work for a consumer finance company which specialises in lending various types of loans to urban customers. When the company receives a loan application, the company has to make a decision for loan approval based on the applicant’s profile. Two types of risks are associated with the bank’s decision:

- If the applicant is likely to repay the loan, then not approving the loan results in a loss of business to the company

- If the applicant is not likely to repay the loan, i.e. he/she is likely to default, then approving the loan may lead to a financial loss for the company


Importing required python libraries useful for analysis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

## Data loading:
Data provided contains information about past loan applicants and whether they ‘defaulted’ or not. Applications that are rejected data is not available in dataset.

In [2]:
# Loading dataset from loan.csv file
df_lender = pd.read_csv('loan.csv')
df_lender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


In [3]:
# Describe data summary
df_lender.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,3.971700e+04,3.971700e+04,39717.000000,39717.000000,39717.000000,39717.000000,3.971700e+04,39717.000000,39717.000000,39717.000000,...,0.0,0.0,0.0,0.0,39020.000000,39678.0,0.0,0.0,0.0,0.0
mean,6.831319e+05,8.504636e+05,11219.443815,10947.713196,10397.448868,324.561922,6.896893e+04,13.315130,0.146512,0.869200,...,NaN,NaN,NaN,NaN,0.043260,0.0,NaN,NaN,NaN,NaN
std,2.106941e+05,2.656783e+05,7456.670694,7187.238670,7128.450439,208.874874,6.379377e+04,6.678594,0.491812,1.070219,...,NaN,NaN,NaN,NaN,0.204324,0.0,NaN,NaN,NaN,NaN
min,5.473400e+04,7.069900e+04,500.000000,500.000000,0.000000,15.690000,4.000000e+03,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN
25%,5.162210e+05,6.667800e+05,5500.000000,5400.000000,5000.000000,167.020000,4.040400e+04,8.170000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN
50%,6.656650e+05,8.508120e+05,10000.000000,9600.000000,8975.000000,280.220000,5.900000e+04,13.400000,0.000000,1.000000,...,NaN,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN
75%,8.377550e+05,1.047339e+06,15000.000000,15000.000000,14400.000000,430.780000,8.230000e+04,18.600000,0.000000,1.000000,...,NaN,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN
max,1.077501e+06,1.314167e+06,35000.000000,35000.000000,35000.000000,1305.190000,6.000000e+06,29.990000,11.000000,8.000000,...,NaN,NaN,NaN,NaN,2.000000,0.0,NaN,NaN,NaN,NaN


In [4]:
# Display shape of dataset
print("Shape of dataframe: ", df_lender.shape)

Shape of dataframe:  (39717, 111)


### Initial observation from dataset
Loan dataset have 111 columns and 39717 user details.
Among all columns only 87 columns have numeric values before manupulating/converting data types.

Now we will proceed forther to investigate how many columns actually valid.

In [5]:
s_null_mean = df_lender.isnull().mean()
print("Number of columns with more then 90% null values :", sum(s_null_mean > 0.9))
print("Number of columns with more then 70% null values :", sum(s_null_mean > 0.6))
print("Number of columns with more then 40% null values :", sum(s_null_mean > 0.3))

Number of columns with more then 90% null values : 56
Number of columns with more then 70% null values : 57
Number of columns with more then 40% null values : 58


As per above output the input dataset have
- 56 columns with more then 90% values are not available.
- 57 columns with more then 60% values are not available..
- 58 columns with more then 30% values are not available.

Now lets get the name and discription of columns which are having more then 30% null values and less then 90% null values.
                                                

In [6]:
list_column_names_90p = s_null_mean.index[~(s_null_mean > 0.9)].tolist()
list_column_names_40p = s_null_mean.index[~(s_null_mean > 0.3)].tolist()
diff_90p_40p = [x for x in list_column_names_90p if x not in list_column_names_40p]
diff_90p_40p

['desc', 'mths_since_last_delinq']

From above output we got the information about columns which have more then 30% and less then 90% of null values are:
 - 'desc' as per data dictonary, Loan description provided by the borrower.
 - 'mths_since_last_delinq' as per data dictionary, The number of months since the borrower's last delinquency.
 Both above parameters are importent for loan process so we can't remove these columns.
 
 Now we can proceed for data cleaning and manipulation of data based on available information.

## Data cleaning:
As per above initial analysis on available data we can remove the columns which are having more then 90% null values.

In [7]:
df_lender = df_lender[list_column_names_90p]
df_lender.shape

(39717, 55)

Now we have 55 columns in dataset after removing invalid columns